# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Programação para Ciência de Dados</font>

## <font color="blue">Pandas Parte III</font>
**Material Produzido por Luis Gustavo Nonato e Bruno Coelho**<br>
**Cemeai - ICMC/USP São Carlos**
---


__Conteúdo:__
- GroupBy
    - Agregação
    - Filtragem
    - Transformação

__Referências__ <br>
- [Pandas: powerful Python data analysis toolkit: Wes McKinney & PyData Devel. Team](https://pandas.pydata.org/pandas-docs/stable/pandas.pdf)
- [http://pandas.pydata.org/pandas-docs/stable/index.html](http://pandas.pydata.org/pandas-docs/stable/index.html)


## Introdução

Frequentemente queremos aplicar transformações e filtragens  em um conjunto de dados a fim de extrair informações relevantes e padrões contidos nos dados. Por exemplo, analisar o salário médio de um grupo de profissionais considerando o sexo e a faixa etária dos profissionais.
Muitas destas transformações podem ser realizadas por meio de uma operação denominada _MapReduce_.

_MapReduce_ é implementada no pacote <font color='blue'>Pandas</font> por meio do método <font color='blue'>groupby</font>.

Em geral, o método <font color='blue'>groupby</font> envolve 3 tarefas:
- __Split__: Divide os dados em subgrupos. Por exemplo, divide os profissionais em subgrupos de acordo com a faixa etária dos profissionais.
- __Apply__: Aplica alguma transformação, agregação ou filtragem para extrair informações de cada subgrupo. Por exemplo, pode-se calcular a média salarial em cada faixa etária dos profissionais.
- __Combine__: Combina os resultados das transformações em um DataFrame ou Série.


$$
\begin{array}{ccccc}
DataFrame & & Split & Apply & Combine\\
\begin{array}{c|c}
C1 & C2 \\ \hline
A & 0 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 5 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 10 \\ \hline
B & 5 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\nearrow \\ \\
\rightarrow \\ \\
\searrow
\end{array} &
\begin{array}{c|c}
A & 0 \\ \hline
A & 5 \\ \hline
A & 10 \\ \\
B & 5 \\ \hline
B & 5 \\ \hline
B & 5 \\ \\
C & 10 \\ \hline
C & 10 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\searrow\\ \\
\rightarrow \\ \\
\nearrow
\end{array} & 
\begin{array}{c|c}
A & 15 \\ \hline
B & 15 \\ \hline
C & 30 
\end{array}
\end{array}
$$

A etapa de "split" divide um conjunto de dados de acordo com algum critério, que pode ser valores das colunas do DataFrame, lista de valores externos ou até mesmo o resultado de uma função. 

Na verdade, o  <font color='blue'>pandas</font> não divide o DataFrame, mas cria uma estrutura que permite operar como se os dados estivessem divididos, evitando o uso excessivo de memória.

Portanto, é importante estar atento, pois o resultado de aplicar o método <font color='blue'>groupby</font> não é um novo DataFrame (ou Serie), mas sim um objeto do tipo `groupby`.
Para visualizar ou acessar os grupos criados pode-se realizar uma redução (que é o resultado do "apply" e "combine") ou utilizar algum método do objeto `groupby`, como por exemplo:
- <font color='blue'>first()</font>: apresenta a primeira linha de cada grupo formado
- <font color='blue'>get_group()</font>: retorna um DataFrame com o conteúdo de um grupo
- <font color='blue'>groups()</font>: retorna um dicionário onde as chaves são os rótulos dos grupos e os valores os índices das linhas onde elementos do grupo ocorrem



In [4]:
# Importando o pacote 'pandas' e 'numpy'
import pandas as pd
import numpy as np

# O comando abaixo carrega o arquivo 'tips.csv' disponível no moodle
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
# Agrupando os dados de acordo com os valores da coluna "sex"

dfgb_sex = df.groupby("sex")

# Perceba que a variável 'df_sex' não é um DataFrame, mas sim 
# um 'DataFrameGroupBy'
print(type(dfgb_sex))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [6]:
# visualizando a primeira linha de cada grupo
print(type(dfgb_sex.first()))  # note que ao invocar o método first() obtemos um DataFrame
print('\n primeira linha cada grupo:\n',dfgb_sex.first())  # note que os rótulos das linhas são
                                                           # os grupos obtidos

<class 'pandas.core.frame.DataFrame'>

 primeira linha cada grupo:
         total_bill   tip smoker  day    time  size
sex                                               
Female       16.99  1.01     No  Sun  Dinner     2
Male         10.34  1.66     No  Sun  Dinner     3


In [8]:
# visualizando as primeiras linhas do grupo 'Male'
print('\n',type(dfgb_sex.get_group('Male')))  # o resultado do get_group é um DataFrame
print('\n primeiras linhas do grupo "Male":\n',
      dfgb_sex.get_group('Male').head())


 <class 'pandas.core.frame.DataFrame'>

 primeiras linhas do grupo "Male":
    total_bill   tip smoker  day    time  size
1       10.34  1.66     No  Sun  Dinner     3
2       21.01  3.50     No  Sun  Dinner     3
3       23.68  3.31     No  Sun  Dinner     2
5       25.29  4.71     No  Sun  Dinner     4
6        8.77  2.00     No  Sun  Dinner     2


In [9]:
# o método 'groups' retorna um dicionário onde as chaves 
# são os rótulos dos grupos formados e os valores são os índices das linhas
# onde os elementos do grupo estão
print('\n',type(dfgb_sex.groups))  # o resutlado de groups é um dicionário
print('\n rótulos dos grupos:\n',dfgb_sex.groups.keys())  # as chaves são os rótulos dos grupos
print('\n indices de alguns elementos do grupo "Female":\n',
      dfgb_sex.groups['Female'][0:5])


 <class 'pandas.io.formats.printing.PrettyDict'>

 rótulos dos grupos:
 dict_keys(['Female', 'Male'])

 indices de alguns elementos do grupo "Female":
 Int64Index([0, 4, 11, 14, 16], dtype='int64')


#### Agrupando de acordo com uma lista externa
O método <font color='blue'>groupby</font> pode agrupar um DataFrame de acordo com uma lista externa (que não é parte do DataFrame). Para isso, a lista deve possuir um número de elementos igual ao número de linhas do DataFrame.

In [18]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=5),
                  'data2': np.random.uniform(low=0,high=1,size=5)})
print(df)

# criando uma lista de 0 e 1 com o mesmo número de linhas do DataFrame
ls = [i for i in np.random.randint(0,2,df.shape[0])]

#lista criada
print(ls) 

  key1 key2     data1     data2
0    a  one  0.284313  0.387097
1    a  two  0.559877  0.473687
2    b  one  0.153237  0.714241
3    b  two  0.068335  0.187249
4    a  one  0.605335  0.736916
[1, 0, 1, 1, 0]


In [19]:
# Agrupando o DataFrame de acordo com a lista 'ls'
gbylist = df.groupby(ls)

print(gbylist.get_group(0))  # linhas onde o valor 0 aparece na lista
print(gbylist.get_group(1))  # linhas onde o valor 1 aparece na lista

  key1 key2     data1     data2
1    a  two  0.559877  0.473687
4    a  one  0.605335  0.736916
  key1 key2     data1     data2
0    a  one  0.284313  0.387097
2    b  one  0.153237  0.714241
3    b  two  0.068335  0.187249


#### Agrupando com múltiplos critérios (indexação hierárquica)
Quando mais que um conjunto de valores é enviado como parâmetro para o <font color='blue'>groupby</font>, o resultado é um agrupamento com índices organizados de forma hierárquica.

In [89]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a','a','b','a'],
                  'key2': ['one','two','one','two','one','two','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=8),
                  'data2': np.random.uniform(low=0,high=1,size=8)})

df

,key1,key2,data1,data2
0,a,one,0.439193,0.936900
1,a,two,0.102908,0.184986
2,b,one,0.045203,0.347324
3,b,two,0.750849,0.059108
4,a,one,0.450183,0.214559
5,a,two,0.510446,0.216753
6,b,two,0.876340,0.629353
7,a,one,0.907657,0.609903


In [20]:
# agrupando com base nos valores das colunas 'key1' e 'key2'
dfh = df.groupby(['key1','key2'])

# imprimindo o conteúdo de cada grupo com um laço for
# 'groupname' corresponde ao índice do grupo, uma tupla neste caso
# 'group' corresponde ao grupo\ própriamente dito
for groupname,group in dfh:
    print('Rotulo do Grupo: ',groupname)
    print(group,'\n')

Rotulo do Grupo:  ('a', 'one')
  key1 key2     data1     data2
0    a  one  0.284313  0.387097
4    a  one  0.605335  0.736916 

Rotulo do Grupo:  ('a', 'two')
  key1 key2     data1     data2
1    a  two  0.559877  0.473687 

Rotulo do Grupo:  ('b', 'one')
  key1 key2     data1     data2
2    b  one  0.153237  0.714241 

Rotulo do Grupo:  ('b', 'two')
  key1 key2     data1     data2
3    b  two  0.068335  0.187249 



#### Agrupando com funções
Uma função pode ser enviada como parâmetro de agrupamento para o <font color='blue'>groupby</font>. Neste caso, a função é aplicada aos rótulos das linhas do DataFrame e o resultado é utilizado como rótulo do grupo.

In [21]:
# constuindo DataFrame com rótulos de linhas e colunas
dfp = pd.DataFrame(data=np.random.randint(low=0, high=10, size=(6,5)),
               columns=['a','b','c','d','e'], 
               index=['Joe','Michel','Steve','Wes','Jim','Travis'])
dfp

,a,b,c,d,e
Joe,9,2,7,7,9
Michel,3,9,2,0,0
Steve,5,8,8,4,9
Wes,0,3,3,9,9
Jim,7,3,3,0,3
Travis,3,2,7,5,2


In [22]:
# agrupando de acordo com o número de caracteres nos rótulos das linhas
gbf = dfp.groupby(lambda x: len(x))

for name, group in gbf:
    print('Grupo: ',name)
    print(group,'\n')

Grupo:  3
     a  b  c  d  e
Joe  9  2  7  7  9
Wes  0  3  3  9  9
Jim  7  3  3  0  3 

Grupo:  5
       a  b  c  d  e
Steve  5  8  8  4  9 

Grupo:  6
        a  b  c  d  e
Michel  3  9  2  0  0
Travis  3  2  7  5  2 



# Transformações e Agregações

Em geral, temos 5 tipos de operações que podem ser aplicadas aos grupos gerados pelo <font color='blue'>groupby</font>:


- __Métodos de agregação__: Combinam várias linhas em um único valor. Exemplos incluem a média, soma e mediana de cada coluna em cada grupo.

- __Métodos de filtragem__: Retornam apenas um subconjunto dos dados originais. 

- __Métodos de tranformação__: Retornam um DataFrame com o mesmo tamanho e índices dos dados originais, mas com valores transformados com base nos grupos.

## Métodos de agregação

Pandas fornece [diversas funções](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) estatísticas de agregação, como <font color='blue'>sum, mean, std, max, min </font>. Pode-se ainda calcular várias agregações simultaneamente com o método <font color='blue'>agg</font> (de "aggregate")

In [23]:
# Importando o pacote 'pandas'
import pandas as pd

# Carregando o arquivo 'tips.csv'
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [173]:
# Obtendo a maior gorjeta dos grupos 'Male' e 'Female'
print("Gorjeta máxima")

# Agrupamos por sexo, depois selecionamos a coluna "tip" (gorjeta)
# e pegamos o máximo dela.
print(df.groupby(["sex"])["tip"].max())


Gorjeta máxima
sex
Female     6.5
Male      10.0
Name: tip, dtype: float64


In [24]:
# Pode-se calcular várias agregações simultaneamente via método "agg"
print("Média e Desvio Padrão das gorjetas em cada grupo:")
df.groupby(["sex"])["tip"].agg([ "mean", "std"])

Média e Desvio Padrão das gorjetas em cada grupo:


,mean,std
sex,,
Female,2.833448,1.159495
Male,3.089618,1.489102


**Importante**: O resultado de uma agregação, transformação ou filtragem  é um novo DataFrame onde os rótulos das linhas são os valores utilizados para realizar o agrupamento. 
Caso queiramos um DataFrame com linhas indexadas com valores $0,1,...$, devemos empregar o método <font color='blue'>reset_index</font>.

In [106]:
# Usando as colunas de agregação como índices
result = df.groupby(["sex"])["tip"].agg(["mean", "std"])
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

            mean       std
sex                       
Female  2.833448  1.159495
Male    3.089618  1.489102 

Rotulos dos grupos:
 Index(['Female', 'Male'], dtype='object', name='sex')

Rotulos das colunas:
 Index(['mean', 'std'], dtype='object')


In [25]:
# Reindexando com reset_index(), os rótulos se tornam uma nova coluna
result = df.groupby(["sex"])["tip"].agg(["mean", "std"]).reset_index()
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

      sex      mean       std
0  Female  2.833448  1.159495
1    Male  3.089618  1.489102 

Rotulos dos grupos:
 RangeIndex(start=0, stop=2, step=1)

Rotulos das colunas:
 Index(['sex', 'mean', 'std'], dtype='object')


Pode-se aplicar agregações diferentes para cada coluna com o uso de dicionário.

In [27]:
print(df.head())
# calculando a média das gorjetas, 
# o valor máximo das refeições 
# e o dia da semana mais frequente (moda) de cada grupo 'Male' e 'Female'
dfm = df.groupby(["sex"]).agg({"tip": "mean", 
                         "total_bill": "max",
                         "day": lambda x: x.mode()})


print('\n',dfm.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

              tip  total_bill   day
sex                               
Female  2.833448       44.30  Thur
Male    3.089618       50.81   Sat


In [114]:
# Pode-se renomear as colunas do DataFrame gerado durante a agregação
# renomeando colunas com agg
print(df.groupby(["sex"]).agg(mean_tip=("tip", "mean"),
                              max_bill=("total_bill", "max"),
                              most_freq_day=("day", lambda x: x.mode())))

        mean_tip  max_bill most_freq_day
sex                                     
Female  2.833448     44.30          Thur
Male    3.089618     50.81           Sat


## Métodos de filtragem

O método mais comum de filtragem é o <font color='blue'>filter </font>que retorna os elementos do grupo que satisfaz uma condição dada. 

In [6]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.989342,0.567076,0.727167
1,c,0.546726,0.909676,0.416302
2,b,0.573113,0.634026,0.885960
3,c,0.702368,0.389789,0.311320
4,a,0.338422,0.638489,0.787344
5,a,0.946119,0.054791,0.013300
6,b,0.035947,0.788968,0.555603
7,a,0.690334,0.520054,0.158044
8,c,0.402787,0.581376,0.698616
9,b,0.681841,0.545786,0.452448


In [7]:
# média geral da coluna 'data1'
data1_mean = df['data1'].mean()
print('media da coluna "data1"\n',data1_mean)

# calculando a média de cada coluna em cada grupo dado pela coluna 'key1'
print('\nmedia de cada grupo\n',df.groupby('key1').mean())

# filtrando os grupos cuja média da coluna 'data1' é maior que a média 
# geral da coluna 'data1'
print('\n grupos cuja média da coluna "data1" é maior que a média geral de "data1"\n')
print(df.groupby('key1').filter(lambda x : x['data1'].mean() > data1_mean))

media da coluna "data1"
 0.5630030147160635

media de cada grupo
          data0     data1     data2
key1                              
a     0.741054  0.445102  0.421464
b     0.430300  0.656260  0.631337
c     0.550627  0.626947  0.475413

 grupos cuja média da coluna "data1" é maior que a média geral de "data1"

  key1     data0     data1     data2
1    c  0.546726  0.909676  0.416302
2    b  0.573113  0.634026  0.885960
3    c  0.702368  0.389789  0.311320
6    b  0.035947  0.788968  0.555603
8    c  0.402787  0.581376  0.698616
9    b  0.681841  0.545786  0.452448


## Métodos de transformação

O método <font color='blue'>transform</font> retorna um DataFrame com o **mesmo número de linhas** que o DataFrame original. A transformação é realizada em cada elemento de cada grupo.

In [8]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.380872,0.913005,0.302514
1,c,0.562794,0.785729,0.132958
2,b,0.444553,0.795973,0.313543
3,c,0.176087,0.886632,0.061810
4,a,0.554507,0.182710,0.259269
5,a,0.747830,0.870483,0.100871
6,b,0.403691,0.443089,0.712434
7,a,0.375358,0.814200,0.623309
8,c,0.024045,0.011379,0.839098
9,b,0.463184,0.437757,0.438303


In [9]:
# Calcula a diferença entre a média do grupo e 
# o elemento do grupo (em cada coluna)
print('média de cada grupo\n',df.groupby("key1").mean())

df[['d1','d2','d3']] = df.groupby("key1").transform(lambda x: x-x.mean())
print('\n',df)

média de cada grupo
          data0     data1     data2
key1                              
a     0.514642  0.695100  0.321491
b     0.437143  0.558940  0.488093
c     0.254309  0.561247  0.344622

   key1     data0     data1     data2        d1        d2        d3
0    a  0.380872  0.913005  0.302514 -0.133770  0.217906 -0.018977
1    c  0.562794  0.785729  0.132958  0.308485  0.224483 -0.211664
2    b  0.444553  0.795973  0.313543  0.007410  0.237033 -0.174551
3    c  0.176087  0.886632  0.061810 -0.078222  0.325385 -0.282812
4    a  0.554507  0.182710  0.259269  0.039865 -0.512389 -0.062222
5    a  0.747830  0.870483  0.100871  0.233188  0.175384 -0.220620
6    b  0.403691  0.443089  0.712434 -0.033451 -0.115851  0.224341
7    a  0.375358  0.814200  0.623309 -0.139283  0.119100  0.301819
8    c  0.024045  0.011379  0.839098 -0.230264 -0.549867  0.494476
9    b  0.463184  0.437757  0.438303  0.026041 -0.121183 -0.049790


O método <font color='blue'>apply</font> aplica uma função em cada grupo e retorna um DataFrame com o resultado da função em cada grupo. Os rótulos das linhas do DataFrame gerado são os identificadores dos grupos. O método <font color='blue'>apply</font> é o mais versátil dos métodos de transformação.

In [32]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.728933,0.394707,0.329381
1,c,0.850069,0.484868,0.765774
2,b,0.723305,0.466161,0.505828
3,c,0.356661,0.298262,0.968979
4,a,0.345557,0.908988,0.264355
5,a,0.951560,0.338186,0.522698
6,b,0.230242,0.995944,0.017758
7,a,0.609910,0.290037,0.903186
8,c,0.508228,0.864183,0.933517
9,b,0.223151,0.456313,0.227825


In [21]:
# agrupando pela coluna 'key1' e calculando a 
# diferença entre o maior e o menor valor de cada coluna em cada grupo
# e retornando a menor diferença

# diferença entre o maior e menor valor em cada coluna em cada grupo
print(df.groupby('key1').apply(lambda x: (x.max()-x.min())))

# menor diferença em cada grupo
max_min = df.groupby('key1').apply(lambda x: (x.max()-x.min()).sort_values()[0])
print(max_min)

         data0     data1     data2        d1        d2        d3
key1                                                            
a     0.372471  0.730295  0.522438  0.372471  0.730295  0.522438
b     0.059493  0.358216  0.398891  0.059493  0.358216  0.398891
c     0.538749  0.875252  0.777288  0.538749  0.875252  0.777288
key1
a    0.372471
b    0.059493
c    0.538749
dtype: float64


**Comentário: MapReduce**

Embora frequentemente chamamos qualquer operação split-apply-combine de _MapReduce_, originalmente esse nome foi dado ao modelo de programação desenvolvido pela Apache.

Sua funcionalidade é a mesma do _GroupBy_ do Pandas, porém foi implementado para operar em grandes volumes de dados utilizando algoritmos distribuidos em uma arquitetura de computação paralela (cluster de computadores), garantindo que cada máquina opere sobre um subconjunto específico dos dados, sendo o padrão para tecnologias de Big Data como [Hadoop](https://hadoop.apache.org/)
